In [56]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Veri Hazırlama

Amacımız, 9 `.csv` dosyasının tamamını tek bir sözlüğe (dict) `data` adında 9 `pandas.DataFrame` olarak yüklemektir. Burada:
- her **anahtar**, csv dosyasının **temizlenmiş adıdır**
- her **değer**, csv dosyasından oluşturulan **DataFrame**tir

```python
data = { 
    'sellers': DataFrame1,
    'orders': DataFrame2,
    ...
    }
```

### 1. `csv_path` değişkenini oluşturun, bu değişken `"csv" klasörü`nün yolunu bir string olarak depolayacaktır

In [57]:
from pathlib import Path

csv_path = Path("~/.workintech/olist/data/csv").expanduser()
csv_path

PosixPath('/Users/gizemtotkanli/.workintech/olist/data/csv')

Yolları kolayca yönetmek için Python'ın `pathlib` modülünü kullanalım. Pathlib çok kullanışlı bir `Path` sınıfı tanımlar.

Bir `Path` örneğini yolumuzu tanımlayan bir string vererek oluşturabiliriz.

In [58]:
file_paths = sorted(csv_path.glob("*.csv"))
file_paths[:3], len(file_paths)

([PosixPath('/Users/gizemtotkanli/.workintech/olist/data/csv/olist_customers_dataset.csv'),
  PosixPath('/Users/gizemtotkanli/.workintech/olist/data/csv/olist_geolocation_dataset.csv'),
  PosixPath('/Users/gizemtotkanli/.workintech/olist/data/csv/olist_order_items_dataset.csv')],
 9)

_`PosixPath`_? Bu, yolun Posix formatında olduğu anlamına gelir. POSIX (Portable Operating System Interface), Unix benzeri işletim sistemleri için uyumluluk ve birlikte çalışabilirliği sağlamak amacıyla standartlaştırılmış API'ler ve kuralların bir setidir. Hem Linux (örneğin WSL'deki Ubuntu) hem de macOS, POSIX uyumludur.

`expanduser()` kullanarak `~` işaretini ana klasörümüzün gerçek mutlak yoluna genişletiyoruz.

Şimdi klasörümüzdeki dosyaları listelemek için `iterdir()` metodunu kullanabiliriz:

### 2. csv dizinindeki tüm csv dosya adlarını içeren `file_names` listesini oluşturun

- Bunun böyle görünmesi gerekir: `file_names = ['olist_sellers_dataset.csv', ....]`
- `file_paths` adresinden başlayabilirsiniz
- Bir `Path` nesnesi `name` adında bir özelliğe sahiptir

In [59]:
file_names = [p.name for p in file_paths]
file_names

['olist_customers_dataset.csv',
 'olist_geolocation_dataset.csv',
 'olist_order_items_dataset.csv',
 'olist_order_payments_dataset.csv',
 'olist_order_reviews_dataset.csv',
 'olist_orders_dataset.csv',
 'olist_products_dataset.csv',
 'olist_sellers_dataset.csv',
 'product_category_name_translation.csv']

### 3. Sözlük anahtarları listesi `key_names` oluşturun
file_names adresinden başlayarak:
- ".csv" sonekini (varsa) kaldırın
- "_dataset.csv" sonekini (varsa) kaldırın
- "olist_" önekini (varsa) kaldırın

<details>
    <summary>- İpucu - </summary>

- `.replace()`
    
- `str` diziler [ ] ile dilimlenebilecek yinelenebilir nesnelerdir
</details>

In [60]:
key_names = [
    name.replace("olist_", "")
        .replace("_dataset.csv", "")
        .replace(".csv", "")
    for name in file_names
]
key_names

['customers',
 'geolocation',
 'order_items',
 'order_payments',
 'order_reviews',
 'orders',
 'products',
 'sellers',
 'product_category_name_translation']

### 4. `data` sözlüğünü oluşturun

```python
data = { 
    'sellers': DataFrame1,
    'orders': DataFrame2,
    'order_items': DataFrame3,
    ...
    }
```
Burada `DataFrame1`, `DataFrame2`, ... gerçek `pandas.DataFrame` nesneleri olmalıdır.

<details>
    <summary>▸ İpucu</summary>

`zip()` metodu iki liste üzerinde yinelemek için çok kullanışlıdır
```python
for (x, y) in zip(['a','b','c'], [1,2,3]):
    print(x,y)

# ('a', 1), ('b', 2), ('c', 3) döndürür
    
```
</details>

In [61]:
import pandas as pd

data = {key: pd.read_csv(path) for key, path in zip(key_names, file_paths)}

sorted(data.keys()), {k: v.shape for k, v in data.items()}

(['customers',
  'geolocation',
  'order_items',
  'order_payments',
  'order_reviews',
  'orders',
  'product_category_name_translation',
  'products',
  'sellers'],
 {'customers': (99441, 5),
  'geolocation': (1000163, 5),
  'order_items': (112650, 7),
  'order_payments': (103886, 5),
  'order_reviews': (99224, 7),
  'orders': (99441, 8),
  'products': (32951, 9),
  'sellers': (3095, 4),
  'product_category_name_translation': (71, 2)})

In [62]:
data["sellers"].head(), list(data["sellers"].columns)

(                          seller_id  seller_zip_code_prefix  \
 0  3442f8959a84dea7ee197c632cb2df15                   13023   
 1  d1b65fc7debc3361ea86b5f14c68d2e2                   13844   
 2  ce3ad9de960102d0677a81f5d0bb7b2d                   20031   
 3  c0f3eea2e14555b6faeea3dd58c1b1c3                    4195   
 4  51a04a8a6bdcb23deccc82b0b80742cf                   12914   
 
          seller_city seller_state  
 0           campinas           SP  
 1         mogi guacu           SP  
 2     rio de janeiro           RJ  
 3          sao paulo           SP  
 4  braganca paulista           SP  ,
 ['seller_id', 'seller_zip_code_prefix', 'seller_city', 'seller_state'])

### 5. `olist/data.py` dosyasındaki `get_data()` metodunu uygulayın

Mantığımızı not defterinden `.py` dosyalarımıza taşımanın zamanı geldi. Bu, bu modül boyunca oluşturacağımız yeni not defterlerinde verileri kolayca yüklememize izin verecek.

Bu modülün klasöründe (bu not defterinin iki seviye yukarısında) `olist/data.py` dosyasını açın ve bu not defterine yazdığınız kodu `get_data()` metoduna taşımaya başlayın. Kodumuzu test etmek için yazdığımız satırları atlayın (`.py` dosyasında listeleri ve DataFrame'leri göstermeye gerek yok).

İşlev çağrıldığında DataFrames içeren sözlüğü döndürmelidir. Bunu csv dosyalarından biriyle deneyip görebilirsiniz:

### Kodunuzu test edin

In [63]:
from pathlib import Path
import pandas as pd


class Olist:
    def get_data(self):
        csv_path = Path("~/.workintech/olist/data/csv").expanduser()

        file_paths = sorted(csv_path.glob("*.csv"))

        def clean_key(path: Path) -> str:
            name = path.name
            name = name.replace("olist_", "")
            name = name.replace("_dataset.csv", "")
            name = name.replace(".csv", "")
            return name

        return {clean_key(p): pd.read_csv(p) for p in file_paths}

In [64]:
from olist.data import Olist

data2 = Olist().get_data()
sorted(data2.keys()), list(data2["sellers"].columns)

(['customers',
  'geolocation',
  'order_items',
  'order_payments',
  'order_reviews',
  'orders',
  'product_category_name_translation',
  'products',
  'sellers'],
 ['seller_city', 'seller_id', 'seller_state', 'seller_zip_code_prefix'])

❓Bu kod parçası makinenizde her yerden çalışmalı, sadece bu not defterinden değil.
- Yeni bir terminal açın
- Ana klasörünüze gidin `cd`
- Bir `ipython` oturumu başlatın
- Yukarıdaki iki satır kodu test edin 👆

In [65]:
from nbresult import ChallengeResult
from olist.data import Olist

data = Olist().get_data()

result = ChallengeResult(
    "get_data",
    keys_len=len(data),
    keys=list(data.keys()),                 # ✅ sıralamayı bozma
    columns=list(data["sellers"].columns),  # ✅ sıralamayı bozma
    vars_used=Olist.get_data.__code__.co_names,
)

result.write()
print(result.check())


============================= test session starts ==============================
platform darwin -- Python 3.12.9, pytest-8.3.4, pluggy-1.5.0 -- /Users/gizemtotkanli/.pyenv/versions/workintech_current/bin/python
cachedir: .pytest_cache
rootdir: /Users/gizemtotkanli/data-data-preparation/tests
plugins: anyio-4.8.0, typeguard-4.4.2
collecting ... collected 3 items

test_get_data.py::TestGetData::test_columns PASSED                       [ 33%]
test_get_data.py::TestGetData::test_keys PASSED                          [ 66%]
test_get_data.py::TestGetData::test_len PASSED                           [100%]

============================== 3 passed in 0.00s ===============================


💯 You can commit your code:

git add tests/get_data.pickle

git commit -m 'Completed get_data step'

git push origin master



🏁 Tebrikler!

💾 Şunları commit ve push etmeyi unutmayın: 
* bu `data_preparation.ipynb` not defteri ve challenge klasöründeki `tests/get_data.pickle` test sonuçları
* ayrıca `/olist` klasöründeki `data.py` dosyası